In [1]:
#enviroonment
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
import os
import tensorflow as tf
import keras.backend.tensorflow_backend as KTF

#important
# GPU use
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)
#keras dynamic growth
KTF.set_session(sess)

Using TensorFlow backend.


In [2]:
#parameters
batch_size = 128
num_classes = 82
epochs = 12
# input image dimensions
img_rows, img_cols = 45,45

In [3]:
import utils
x_train, y_train, x_test, y_test = utils.load_dataset()

In [4]:
#prepare data
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

# x_train = x_train.astype('float32')
# x_test = x_test.astype('float32')
# x_train /= 255
# x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (54078, 45, 45, 1)
54078 train samples
13505 test samples


In [5]:
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [6]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

In [7]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [8]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

Train on 54078 samples, validate on 13505 samples
Epoch 1/12
54078/54078 [==============================] - 30s 562us/step - loss: 1.9349 - acc: 0.5138 - val_loss: 0.8866 - val_acc: 0.7622
Epoch 2/12
54078/54078 [==============================] - 28s 526us/step - loss: 0.9660 - acc: 0.7306 - val_loss: 0.6699 - val_acc: 0.8192
Epoch 3/12
54078/54078 [==============================] - 28s 523us/step - loss: 0.7142 - acc: 0.7939 - val_loss: 0.5758 - val_acc: 0.8458
Epoch 4/12
54078/54078 [==============================] - 28s 521us/step - loss: 0.5685 - acc: 0.8337 - val_loss: 0.4967 - val_acc: 0.8626
Epoch 5/12
54078/54078 [==============================] - 28s 521us/step - loss: 0.4912 - acc: 0.8532 - val_loss: 0.4815 - val_acc: 0.8756
Epoch 6/12
54078/54078 [==============================] - 28s 521us/step - loss: 0.4254 - acc: 0.8720 - val_loss: 0.4536 - val_acc: 0.8788
Epoch 7/12
54078/54078 [==============================] - 28s 521us/step - loss: 0.3715 - acc: 0.8850 - val_loss: 0.

In [11]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 43, 43, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 41, 41, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 20, 20, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 20, 20, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 25600)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               3276928   
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
__________

In [16]:
model.save('Symbol_Net_model_epoch_12_.h5')  